# Building a dataset with International Mathematical Olympiad results and Macroeconomics indicators

The aim of this note book is to produce a dataframe made of data of countries on International Mathematical Olympiad results and Macroeconomics indicators.

## Installing and importing Packages

Install relevant packages ( Transform below cell to code )

In [ ]:
pip install numpy pandas matplotlib statsmodels wbgapi itables

Import relevant packages

In [1]:
from matplotlib import pyplot as plt  # to draw
import requests  # get pages from urls
import pandas as pd  # library to handle datasets
from itertools import islice
import wbgapi as wb  # Woldbank API for macro data
import re
from itables import show  # for pretty display of dataframes
import glob

## Init

In [2]:
LOCAL = True

## Global function

In [3]:
def get_df_from_url(url):
    html = requests.get(url).content
    df_list = pd.read_html(html)
    return df_list

## Get Imo country codes

In [5]:
def get_imo_country_codes():
    filename = "df_imo_country_code.cvs"
    link = "https://www.imo-official.org/countries.aspx"
    ### Read from file
    if LOCAL and glob.glob(filename):
        df_imo_country_code = pd.read_csv(filename)
    ### Or download
    else:
        print("downloading")
        dfs = get_df_from_url(link)
        df_imo_country_code = dfs[0]
        df_imo_country_code.to_csv(filename, index=False)
    return df_imo_country_code

In [6]:
df_imo_country_code = get_imo_country_codes()

In [7]:
df_imo_country_code.head()

,Code,Country,Contact,National MO site,IMO Host
0,AFG,Afghanistan,Onder Akkusci <onderakkuscu@gmail.com>,NaN,NaN
1,ALB,Albania,Adrian Naço <nacosotir@gmail.com>,https://www.ama.org.al,NaN
2,ALG,Algeria,Abdelhafid Hadjsadok - Abdallah Rahmani <olymp...,NaN,NaN
3,AGO,Angola,Gabriel Boaventura <gabrielboaventura1968@gmai...,NaN,NaN
4,ARG,Argentina,Patricia Fauring <fauringpatricia@gmail.com>,http://www.oma.org.ar/,"1997, 2012"


## Get Imo Results

In [7]:
def get_imo_points():
    filename = "df_imo_points.csv"
    link = "https://www.imo-official.org/country_team_r.aspx?code={code}"
    df_imo_points = None
    ### Read from file
    if LOCAL and glob.glob(filename):
        df_imo_points = pd.read_csv(filename)
    ### Or download
    else:
        print("downloading...")
        data = {}
        for code in df_imo_country_code["Code"]:
            try:
                data[code] = get_df_from_url(
                    f"https://www.imo-official.org/country_team_r.aspx?code={code}"
                )
            except Exception as e:
                print("code, error", code, e)
        imo_points = []
        for code in data:
            df = data[code][0]
            df.columns = ["imo_" + "_".join(c) for c in df.columns]
            df["imo_code"] = code
            imo_points.append(df)
        df_imo_points = pd.concat(imo_points)
        df_imo_points.to_csv(filename, index=False)

    return df_imo_points  # ,data

In [8]:
df_imo_points = get_imo_points()
df_imo_points

,imo_Year_Year,imo_Team size_All,imo_Team size_M,imo_Team size_F,imo_P1_P1,imo_P2_P2,imo_P3_P3,imo_P4_P4,imo_P5_P5,imo_P6_P6,...,imo_Awards_B,imo_Awards_HM,imo_Leader_Leader,imo_Deputy leader_Deputy leader,imo_Data quality_S,imo_Data quality_N,imo_Data quality_M,imo_Data quality_G,imo_code,imo_P7_P7
0,2024,6,5.0,1.0,19.0,2.0,0.0,29.0,1.0,0.0,...,1.0,3.0,Erind Bedalli,Bruno Bajo,•,•,•,•,ALB,NaN
1,2023,6,5.0,1.0,32.0,10.0,0.0,5.0,1.0,0.0,...,0.0,4.0,Adrian Naço,Enkel Hysnelaj,•,•,•,•,ALB,NaN
2,2022,6,4.0,2.0,19.0,8.0,0.0,31.0,4.0,0.0,...,0.0,3.0,Adrian Naço,Elton Bojaxhiu,•,•,•,•,ALB,NaN
3,2021,6,5.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,Adrian Naço,Enkel Hysnelaj,•,•,•,•,ALB,NaN
4,2020,6,4.0,2.0,26.0,7.0,1.0,5.0,1.0,0.0,...,1.0,2.0,Adrian Naço,Enkel Hysnelaj,•,•,•,•,ALB,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3775,1965,8,8.0,NaN,26.0,20.0,15.0,33.0,18.0,25.0,...,2.0,NaN,Milica Ilić-Djajović,Vladimir Mićić,•,•,•,•,YUG,NaN
3776,1964,8,7.0,1.0,34.0,28.0,36.0,1.0,26.0,30.0,...,1.0,NaN,Milica C. Dajević,M. Stojanović,NaN,•,•,•,YUG,NaN
3777,1963,8,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,Milica Dajović,M. Marjanović,NaN,•,•,•,YUG,NaN
3778,2014,6,6.0,NaN,1.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,Gift Tapedzesa,Gerald Nqobile Mpitanyanga,•,•,•,•,ZWE,NaN


In [9]:
def get_imo_results_modify(df_imo_points):
    filename = "df_imo_results.csv"
    df_imo_results = None
    ### Read from file
    if LOCAL and glob.glob(filename):
        df_imo_results = pd.read_csv(filename)
    ### Or build
    else:
        print("Building...")
        df_imo_points["year"] = df_imo_points["imo_Year_Year"]
        df_imo_points["imo_total"] = df_imo_points["imo_Total_Total"]
        df_imo_points = df_imo_points[df_imo_points.year >= 1994]
        df_imo_points = df_imo_points["year imo_code imo_total".split()]
        df_imo_results = pd.merge(
            df_imo_points,
            df_imo_country_code,
            left_on=["imo_code"],
            right_on=["Code"],
            how="left",
        )["year 	imo_code 	imo_total	Country".split()]
        df_imo_results.rename(columns={"Country": "country"}, inplace=True)
        df_imo_results.to_csv(filename, index=False)
    return df_imo_results

In [10]:
df_imo_results = get_imo_results_modify(df_imo_points)
df_imo_results

,year,imo_code,imo_total,country
0,2024,ALB,51.0,Albania
1,2023,ALB,48.0,Albania
2,2022,ALB,62.0,Albania
3,2021,ALB,11.0,Albania
4,2020,ALB,40.0,Albania
...,...,...,...,...
2916,1997,YUG,125.0,Yugoslavia
2917,1996,YUG,87.0,Yugoslavia
2918,1995,YUG,154.0,Yugoslavia
2919,2014,ZWE,5.0,Zimbabwe


## Get all country codes from WB

In [11]:
def get_country_codes_from_wb():
    df_wb_countries = None
    filename = "df_imo_wb_codes.csv"
    ### Read from file
    if LOCAL and glob.glob(filename):
        df_wb_countries = pd.read_csv(filename)
    ### Or download
    else:
        print("Downloading...")
        url = "https://wits.worldbank.org/WITS/wits/WITSHELP/Content/Codes/Country_Codes.htm"
        dfs = get_df_from_url(url)
        df_wb_countries = dfs[-1]
        df_wb_countries.to_csv(filename, index=False)
    return df_wb_countries

In [12]:
df_wb_countries = get_country_codes_from_wb()
df_wb_countries.columns = df_wb_countries.iloc[1, :]
df_wb_countries = df_wb_countries.iloc[2:, :]
df_wb_countries.head()

1,Country Name,ISO3,Code
2,Afghanistan,AFG,004
3,Albania,ALB,008
4,Algeria,DZA,012
5,American Samoa,ASM,016
6,Andorra,AND,020


## Get imo results with wb codes

In [13]:
df_imo_results_with_wb_codes = df_imo_results.merge(
    df_wb_countries, left_on="country", right_on="Country Name"
)

In [14]:
df_imo_results_with_wb_codes.rename(
    columns={
        "country": "imo_country_name",
        "Country Name": "wb_country_name",
        "ISO3": "wb_iso_code",
        "Code": "wb_nb_code",
    },
    inplace=True,
)
df_imo_results_with_wb_codes

,year,imo_code,imo_total,imo_country_name,wb_country_name,wb_iso_code,wb_nb_code
0,2024,ALB,51.0,Albania,Albania,ALB,008
1,2023,ALB,48.0,Albania,Albania,ALB,008
2,2022,ALB,62.0,Albania,Albania,ALB,008
3,2021,ALB,11.0,Albania,Albania,ALB,008
4,2020,ALB,40.0,Albania,Albania,ALB,008
...,...,...,...,...,...,...,...
2484,1997,YUG,125.0,Yugoslavia,Yugoslavia,SER,891
2485,1996,YUG,87.0,Yugoslavia,Yugoslavia,SER,891
2486,1995,YUG,154.0,Yugoslavia,Yugoslavia,SER,891
2487,2014,ZWE,5.0,Zimbabwe,Zimbabwe,ZWE,716


## Getting economic Data

In [15]:
def get_econ_data():
    dfs_econ = {}
    filenames = "GDP.csv POP.csv SURF.csv".split()
    codes = dict(
        zip(filenames, """NY.GDP.PCAP.PP.CD SP.POP.TOTL AG.SRF.TOTL.K2""".split())
    )
    for filename in filenames:
        if LOCAL and glob.glob(filename):
            dfs_econ[filename[:-4]] = pd.read_csv(filename)
        else:
            dfs_econ[filename[:-4]] = wb.data.DataFrame(codes[filename])
            dfs_econ[filename[:-4]]["economy"] = dfs_econ[filename[:-4]].index
            dfs_econ[filename[:-4]].to_csv(filename, index=False)

    return dfs_econ

In [16]:
dfs_econ = get_econ_data()
dfs_econ["GDP"].head()

,YR1960,YR1961,YR1962,YR1963,YR1964,YR1965,YR1966,YR1967,YR1968,YR1969,...,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023,economy
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35972.866445,36117.508211,37524.928478,39287.019517,39110.276190,28976.463550,35696.309372,41649.450792,44967.344513,ABW
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3456.365448,3551.069323,3703.886605,3648.310911,3742.848955,3629.508004,3907.951637,4229.682100,4374.229532,AFE
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2284.075848,2213.181441,2335.795862,2432.276701,2583.485332,2561.981761,2144.166570,2122.995815,2211.280635,AFG
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4052.733797,3996.864267,4063.185050,4198.081042,4454.156038,4441.937338,4698.926894,5107.272939,5343.468529,AFW
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7119.726443,6843.735897,6992.728834,7347.799936,7528.382418,6450.749946,7408.126591,7924.888806,8040.702450,AGO


## Add economic data

In [28]:
LOCAL = 1

In [29]:
def add_economic_data():
    df_imo_macro = None
    filename = "df_imo_macro.cvs"
    ### Read from file
    if LOCAL and glob.glob(filename):
        print("Loading df_imo_macro.cvs...")
        df_imo_macro = pd.read_csv(filename)
    ### Or process
    else:
        print("Processing...")
        dfs = {}
        for s in "GDP POP SURF".split():
            dfs[s] = {"wb_code": [], "year": [], s.lower(): []}
            df = dfs_econ[s]
            for i in df.index:
                code = df.economy.loc[i]
                for col in df.columns[:-1]:
                    year = int(col[2:])
                    if year < 1994:
                        continue
                    dfs[s]["wb_code"].append(code)
                    dfs[s]["year"].append(year)
                    dfs[s][s.lower()].append(df[col].loc[i])

        dfs = {key: pd.DataFrame(val) for key, val in dfs.items()}
        df_economic_data = pd.DataFrame()
        df_economic_data["wb_code"] = dfs["GDP"].wb_code
        df_economic_data["year"] = dfs["GDP"].year
        df_economic_data["gdp"] = dfs["GDP"].gdp
        df_economic_data["pop"] = dfs["POP"]["pop"]
        df_economic_data["surf"] = dfs["SURF"].surf

        df_imo_macro = pd.merge(
            df_economic_data,
            df_imo_results_with_wb_codes,
            how="outer",
            left_on=["wb_code", "year"],
            right_on=["wb_iso_code", "year"],
        )
        df_imo_macro.imo_total.fillna(0, inplace=True)
        df_imo_macro.imo_code.fillna("", inplace=True)
        df_imo_macro["wb_country_name"].fillna(
            "", inplace=True
        )  # = df_imo_macro.wb_code.apply(lambda x: codes_dic_reverse.get(x,"NA"))
        df_imo_macro.fillna("", inplace=True)
        df_imo_macro.to_csv(filename, index=False)
    return df_imo_macro

In [30]:
df_imo_macro = add_economic_data()
df_imo_macro

Loading df_imo_macro.cvs...


,wb_code,year,gdp,pop,surf,imo_code,imo_total,imo_country_name,wb_country_name,wb_iso_code,wb_nb_code
0,ABW,1994,25791.043178,77595.0,180.0,NaN,0.0,NaN,NaN,NaN,NaN
1,ABW,1995,26254.742803,79805.0,180.0,NaN,0.0,NaN,NaN,NaN,NaN
2,ABW,1996,26004.495628,83021.0,180.0,NaN,0.0,NaN,NaN,NaN,NaN
3,ABW,1997,27240.802789,86301.0,180.0,NaN,0.0,NaN,NaN,NaN,NaN
4,ABW,1998,27412.755194,88451.0,180.0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8136,ZWE,2019,3211.471913,15271368.0,390760.0,NaN,0.0,NaN,NaN,NaN,NaN
8137,ZWE,2020,3510.594036,15526888.0,390760.0,NaN,0.0,NaN,NaN,NaN,NaN
8138,ZWE,2021,3184.785451,15797210.0,390760.0,NaN,0.0,NaN,NaN,NaN,NaN
8139,ZWE,2022,3560.043273,16069056.0,390760.0,NaN,0.0,NaN,NaN,NaN,NaN


In [35]:
df_imo_macro.fillna("", inplace=True)

/tmp/ipykernel_22283/2154245071.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_imo_macro.fillna("",inplace=True)


In [ ]:
country_names = {}
for i in range(df_wb_countries.shape[0]):
    name = df_wb_countries["Country Name"].iloc[i]
    code = df_wb_countries["ISO3"].iloc[i]
    if code and (not code in country_names) and name:
        country_names[code] = name

# country_names

In [51]:
df_imo_macro["wb_country_name"] = df_imo_macro.wb_code.apply(
    lambda x: country_names.get(x, "")
)

## Add hystorical data

In [52]:
def get_indep_data():
    df_indep = None
    filename = "df_indep.csv"
    ### Read from file
    if LOCAL and glob.glob(filename):
        df_indep = pd.read_csv(filename)
    ### Or download
    else:
        print("Downloading...")
        url = "https://en.wikipedia.org/wiki/List_of_national_independence_days"
        df_indeps = get_df_from_url(url)
        df_indep = df_indeps[-1]
        df_indep.to_csv(filename, index=False)
    return df_indep

In [53]:
df_indep = get_indep_data()
df_indep

,Country,Name of holiday,Date of holiday,Year of event,Independence from,Event commemorated and notes
0,Afghanistan,Independence Day,19 August,1919,United Kingdom,Anglo-Afghan Treaty of 1919 or Treaty of Rawal...
1,Afghanistan,Liberation Day,15 February,1989,Soviet Union,Soviet withdrawal from Afghanistan
2,Albania,Independence Day,28 November,1912,Ottoman Empire,Albanian Declaration of Independence. During t...
3,Algeria,Independence Day,5 July,1962,France,Algeria gained independence following the Alge...
4,Angola,Independence Day,11 November,1975,Portugal,"The Alvor Agreement, signed on 15 January 1975..."
...,...,...,...,...,...,...
201,Venezuela,Independence Day,5 July,1811,Spanish Empire,Venezuelan Declaration of Independence.
202,Vietnam,National Day,2 September,1945,Empire of Japan France,Proclamation of Independence of the Democratic...
203,Yemen,Independence Day,30 November,1967,United Kingdom,Declaration of independence as Kingdom of Yemen
204,Zambia,Independence Day,24 October,1964,United Kingdom,Effective date of the Zambia Independence Act ...


In [54]:
def get_set_indep_after(year=1955):
    df_g = df_indep.groupby(["Country"])
    indep = {}

    def after_year(strg, year=year):
        y = 0
        try:
            y = int(strg[:4])
        except:
            print(strg)
        return y > year

    for i in df_indep.index[:-1]:
        country = df_indep.iloc[i].Country
        date = df_indep.iloc[i, 3]
        if country not in indep:
            indep[country] = date
    indep_after_year = {
        country for country, date in indep.items() if after_year(date, year)
    }
    return indep_after_year

In [68]:
def get_df_imo_macro_hist():
    df_imo_macro_hist = None
    filename = "df_imo_macro_hist.csv"
    indep_after_year = get_set_indep_after()
    ### Read from file
    if LOCAL and glob.glob(filename):
        df_imo_macro_hist = pd.read_csv(filename)
    ### Or download
    else:
        print("Building...")
        df_imo_macro_hist = df_imo_macro.copy()
        df_imo_macro_hist["indep_after_1955"] = df_imo_macro_hist[
            "wb_country_name"
        ].apply(lambda x: int(x in indep_after_year))
        df = df_imo_macro_hist
        selection = df[df["pop"].isna()].index
        df = df.drop(selection, axis=0)
        selection = df[df["gdp"].isna()].index
        df = df.drop(selection, axis=0)
        selection = df[df["surf"].isna()].index
        df = df.drop(selection, axis=0)
        df.fillna("", inplace=True)
        df_imo_macro_hist = df_imo_macro_hist[
            [
                "wb_country_name",
                "wb_code",
                "year",
                "gdp",
                "pop",
                "surf",
                "imo_total",
                "indep_after_1955",
            ]
        ]
        df_imo_macro_hist.to_csv(filename, index=False)
    return df_imo_macro_hist

In [69]:
df_imo_macro_hist = get_df_imo_macro_hist()
df_imo_macro_hist.head()

Building...


,wb_country_name,wb_code,year,gdp,pop,surf,imo_total,indep_after_1955
0,Aruba,ABW,1994,25791.043178,77595.0,180.0,0.0,0
1,Aruba,ABW,1995,26254.742803,79805.0,180.0,0.0,0
2,Aruba,ABW,1996,26004.495628,83021.0,180.0,0.0,0
3,Aruba,ABW,1997,27240.802789,86301.0,180.0,0.0,0
4,Aruba,ABW,1998,27412.755194,88451.0,180.0,0.0,0


In [71]:
show(df_imo_macro_hist, maxBytes=0)

# End processing